# 1. IMPORT REQUIRED PACKAGES

In [419]:
#%pip install --upgrade kaleido

In [420]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import json
import os
import re
import statsmodels
import yfinance as yf
import plotly.express as px
import re
import yfinance as yf
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path


In [421]:
#For Seaborn plots red-blue colour palette
KES_palette = ["#B80F0A", "#E63946", "#D62828", "#780000", "#F4A261", "#E76F51", "#2A9D8F", "#264653", "#1D3557", "#457B9D", "#A8DADC"]
sns.set_palette(KES_palette)

# 2. PROJECT PATHS

In [422]:
BASE_DIR = Path(".").resolve()
PROJ_DIR = Path("/Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis")
IN_DIR = PROJ_DIR / "inputs"
OUT_DIR = PROJ_DIR / "outputs"
FIG_DIR = PROJ_DIR / "figures"
CFG_DIR = PROJ_DIR / "config"
# ...existing code..

for p in [IN_DIR, OUT_DIR, FIG_DIR, CFG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print(f"Project dir: {PROJ_DIR}")
print(f"Inputs dir:  {IN_DIR}")
print(f"Outputs dir: {OUT_DIR}")
print(f"Figures dir: {FIG_DIR}")
print(f"Config dir:  {CFG_DIR}")

Project dir: /Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis
Inputs dir:  /Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis/inputs
Outputs dir: /Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis/outputs
Figures dir: /Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis/figures
Config dir:  /Users/kruri/Desktop/Kent Economics Society/EV_insurance/Data Analysis/config


# 2. IMPORT DATASETS

## 2.1 Vehicles Data and other Government outputs.

In [423]:
df_veh = pd.read_csv(IN_DIR / "df_VEH0120_GB.csv", low_memory=False)
df_veh.info()
df_fuel = pd.read_csv(IN_DIR / "fuel_data.csv", low_memory=False)

# config files
with open(CFG_DIR / "brand_Country.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

df_brand = pd.DataFrame(lines, columns=["raw"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240631 entries, 0 to 240630
Data columns (total 87 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   BodyType       240631 non-null  object
 1   Make           240631 non-null  object
 2   GenModel       240629 non-null  object
 3   Model          240631 non-null  object
 4   Fuel           240631 non-null  object
 5   LicenceStatus  240631 non-null  object
 6   2025Q1         240631 non-null  int64 
 7   2024Q4         240631 non-null  int64 
 8   2024Q3         240631 non-null  int64 
 9   2024Q2         240631 non-null  int64 
 10  2024Q1         240631 non-null  int64 
 11  2023Q4         240631 non-null  int64 
 12  2023Q3         240631 non-null  int64 
 13  2023Q2         240631 non-null  int64 
 14  2023Q1         240631 non-null  int64 
 15  2022Q4         240631 non-null  int64 
 16  2022Q3         240631 non-null  int64 
 17  2022Q2         240631 non-null  int64 
 18  2022

### 2.1.2 Cleaning and Sorting Vehicles Data


In [424]:
quarter_cols = [c for c in df_veh.columns if c.startswith("20") and "Q" in c]

# 3. Melt to long format
df_long = df_veh.melt(
    id_vars=["BodyType", "Make", "GenModel", "Model", "Fuel", "LicenceStatus"],
    value_vars=quarter_cols,
    var_name="Quarter",
    value_name="Registrations"
)

# 4. Extract Year and Quarter number and keep cars
df_long["Year"] = df_long["Quarter"].str.extract(r"(20\d{2})").astype(int)
df_long["QuarterNo"] = df_long["Quarter"].str.extract(r"Q(\d)").astype(int)
df_long = df_long[df_long["BodyType"] == "Cars"]
df_long = df_long.drop(columns=["QuarterNo","GenModel","Model","Quarter"])
# Group by Year × Make (collapse simple duplicates by normalising Make)
df_long["Make"] = (df_long["Make"].astype(str)
                   .str.replace("\xa0", " ", regex=False)
                   .str.strip().str.upper()
                   .str.replace(r"\s+", " ", regex=True))



#Fuel type

df_long["Fuel Group"] = df_long["Fuel"].str.strip().str.lower().apply(
    lambda f:
        "Electric" if any(e in f for e in [
            "electric", "battery", "plug-in", "fuel cell"
        ])
        else "Petrol" if "petrol" in f and "electric" not in f
        else "Diesel" if "diesel" in f and "electric" not in f
        else "Gas" if "gas" in f
        else "Other Fuel Types"
)

# Brand origin
df_brand["raw"] = (
    df_brand["raw"]
    .str.replace("\xa0", " ", regex=False)          # NBSP -> space
    .str.replace(r"\s+", " ", regex=True)           # collapse spaces
    .str.replace(r"–|—|-", "—", regex=True)         # any dash -> em dash
    .str.strip()
)

parts = df_brand["raw"].str.split("—", n=3, expand=True)
# If some lines are short/long, pad or trim to 4 columns
if parts.shape[1] < 4:
    parts = parts.reindex(columns=range(4))
parts = parts.apply(lambda c: c.fillna("").str.strip())

df_brand[["Make", "Country", "ParentCompany", "ParentCountry"]] = parts.iloc[:, :4]
df_brand.drop(columns=["raw"])
# Attaching Country of Origin to dataset

df_long = df_long.merge(df_brand[["Make", "ParentCountry"]], on="Make", how="left")

df_veh_raw = df_long.copy()
df_veh_raw.info()
pd.DataFrame(df_veh_raw['Make'].unique(), columns=['Make']).to_excel(
    OUT_DIR / "unique_makes.csv", index=False
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10256250 entries, 0 to 10256249
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   BodyType       object
 1   Make           object
 2   Fuel           object
 3   LicenceStatus  object
 4   Registrations  int64 
 5   Year           int64 
 6   Fuel Group     object
 7   ParentCountry  object
dtypes: int64(2), object(6)
memory usage: 626.0+ MB


In [425]:
df_veh_raw

,BodyType,Make,Fuel,LicenceStatus,Registrations,Year,Fuel Group,ParentCountry
0,Cars,ABARTH,Petrol,Licensed,15,2025,Petrol,Netherlands
1,Cars,ABARTH,Petrol,SORN,4,2025,Petrol,Netherlands
2,Cars,ABARTH,Petrol,Licensed,25,2025,Petrol,Netherlands
3,Cars,ABARTH,Petrol,SORN,3,2025,Petrol,Netherlands
4,Cars,ABARTH,Petrol,Licensed,958,2025,Petrol,Netherlands
...,...,...,...,...,...,...,...,...
10256245,Cars,ZETOR,Petrol,Licensed,0,2000,Petrol,Czech Republic
10256246,Cars,ZHENHUA,Petrol,Licensed,0,2000,Petrol,China
10256247,Cars,ZHENHUA,Diesel,Licensed,0,2000,Diesel,China
10256248,Cars,ZHENHUA,Petrol,SORN,0,2000,Petrol,China


In [426]:
df_brand

,raw,Make,Country,ParentCompany,ParentCountry
0,ABARTH — Italy — Stellantis N.V. — Netherlands,ABARTH,Italy,Stellantis N.V.,Netherlands
1,AC — United Kingdom — AC Cars Ltd — United Kin...,AC,United Kingdom,AC Cars Ltd,United Kingdom
2,AC (ELECTRIC) — United Kingdom — AC Cars Ltd —...,AC (ELECTRIC),United Kingdom,AC Cars Ltd,United Kingdom
3,"ACCESS — Japan — Access Motor Co., Ltd — Japan",ACCESS,Japan,"Access Motor Co., Ltd",Japan
4,"ACURA — Japan — Honda Motor Co., Ltd. — Japan",ACURA,Japan,"Honda Motor Co., Ltd.",Japan
...,...,...,...,...,...
463,YUAN — China — BYD Auto (Yuan model range) — C...,YUAN,China,BYD Auto (Yuan model range),China
464,ZASTAVA — Serbia — Zastava Automobiles (Fiat/S...,ZASTAVA,Serbia,Zastava Automobiles (Fiat/Serbian state heritage),Serbia
465,ZENOS — United Kingdom — Zenos Cars Ltd. — Uni...,ZENOS,United Kingdom,Zenos Cars Ltd.,United Kingdom
466,ZETOR — Czech Republic — Zetor Tractors a.s. —...,ZETOR,Czech Republic,Zetor Tractors a.s.,Czech Republic


## 2.2 Stock Market Data

# 3. Data Analysis

## 3.1 Features

### 3.1.1 Pie Chart of Types of Vehicles

In [427]:
df_veh_raw_2020 = df_veh_raw[df_veh_raw["Year"] >= 2000]
distinct_names_array = df_veh_raw_2020['Fuel Group'].unique()
counts,values = pd.Series(df_veh_raw_2020['Make']).value_counts().values, pd.Series(df_veh_raw_2020['Fuel Group']).value_counts().index
df_results_2020 = pd.DataFrame(list(zip(values,counts)),columns=["Fuel Group","Count"])
df_results_2020
data_2024= df_results_2020.nlargest(n=10,columns="Count")


fig = px.pie(data_2024, values='Count', names='Fuel Group', title='Distribution of EV Registrations by Make (2024 onwards)', color_discrete_sequence=KES_palette)
#plt.pie(data_2024['Count'], labels=data_2024['Fuel Type'], autopct='%1.1f%%', startangle=140, colors=KES_palette)
#DOWNLOAD FIGURE
fig.write_image(FIG_DIR / "pie_chart_fuel_group_registrations_by_make_2024.pdf")


### 3.1.2 Pie Chart of Countries in the EV Space

In [428]:
#import plotly.express as px

# --- Filter electric vehicles ---
df_ev = df_veh_raw[df_veh_raw["Fuel Group"] == "Electric"].copy()
df_ev = df_veh_raw[df_veh_raw["Fuel"] == "Battery electric"].copy()

df_ev = df_ev[df_ev["Year"] >= 2020].copy()

# --- Group by parent country ---
country_counts = (
    df_ev.groupby("ParentCountry", as_index=False)["Registrations"]
         .sum()
)

# --- Replace countries <5% with "Other" and re-aggregate ---
threshold = 0.05 * country_counts["Registrations"].sum()
country_counts["ParentCountry"] = country_counts.apply(
    lambda r: r["ParentCountry"] if r["Registrations"] >= threshold else "Other",
    axis=1
)
country_counts = (
    country_counts.groupby("ParentCountry", as_index=False)["Registrations"]
                  .sum()
                  .sort_values("Registrations", ascending=False)
)
# --- Create pie chart ---
fig = px.pie(
    country_counts,
    values="Registrations",
    names="ParentCountry",
    title="Distribution of Countries in the EV Space",
    hole=0,  # 0.4 -> donut style
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.write_image(FIG_DIR / "pie_chart_Country group.pdf")

### 3.1.3 Line graph: EV registrations by parent country over time

In [429]:
# --- Line graph: EV registrations by parent country over time ---
df_ev = df_ev[df_ev["Year"] >= 2020].copy()

country_trend = (
    df_ev.groupby(["Year", "ParentCountry"], as_index=False)["Registrations"]
         .sum()
         .sort_values(["ParentCountry", "Year"])
)

fig = px.line(
    country_trend,
    x="Year",
    y="Registrations",
    color="ParentCountry",
    title="EV Registrations Over Time by Parent Country",
    markers=True,
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.write_image(FIG_DIR / "line_Country_trend.pdf")

## 4 Economic Metrics


### 4.0.1 Setup

In [430]:
# Filter out 2025
df = df_veh_raw[df_veh_raw["Year"] < 2025].copy()


## 4. Metrics

### 4.1.1 Annual Growth Rate of Total Car Registrations

In [431]:
annual_total = (
    df.groupby("Year", as_index=False)["Registrations"]
      .sum()
      .sort_values("Year")
)
annual_total["GrowthRate_%"] = annual_total["Registrations"].pct_change() * 100
annual_total
# --- Base figure with bars for Registrations ---
fig = go.Figure()

fig.add_trace(go.Bar(
    x=annual_total["Year"],
    y=annual_total["Registrations"],
    name="Registrations",
    marker_color="steelblue",
    opacity=0.7,
))

# --- Add growth rate line (secondary axis) ---
fig.add_trace(go.Scatter(
    x=annual_total["Year"],
    y=annual_total["GrowthRate_%"],
    name="Growth Rate (%)",
    mode="lines+markers",
    yaxis="y2",
    line=dict(color="crimson", width=2)
))

# --- Layout ---
fig.update_layout(
    title="Annual Vehicle Registrations and Year-on-Year Growth",
    xaxis_title="Year",
    yaxis=dict(title="Registrations"),
    yaxis2=dict(
        title="Growth Rate (%)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    template="plotly_white",
    legend=dict(x=0.02, y=0.98),
    width=850, height=500
)

# --- Save ---
fig.write_image(FIG_DIR / "annual_total_growth.pdf")

fig.show()

### 4.1.2 Market Share by Fuel Group

In [432]:
fuel_share = (
    df.groupby(["Year", "Fuel Group"], as_index=False)["Registrations"]
      .sum()
)

fuel_share["MarketShare_%"] = (
    fuel_share.groupby("Year")["Registrations"]
              .transform(lambda x: 100 * x / x.sum())
)

fuel_share.sort_values(["Year", "MarketShare_%"], ascending=[True, False])

# --- Sort data for clarity ---
fuel_share = fuel_share.sort_values(["Year", "MarketShare_%"], ascending=[True, False])

# --- Line chart: each fuel group over time ---
fig = px.line(
    fuel_share,
    x="Year",
    y="MarketShare_%",
    color="Fuel Group",
    markers=True,
    title="Market Share of Fuel Types Over Time",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Market Share (%)",
    legend_title="Fuel Group",
    template="plotly_white",
    width=850,
    height=500
)

# --- Save outputs ---
fig.write_html(FIG_DIR / "fuel_share_trends.html")
fig.write_image(FIG_DIR / "fuel_share_trends.pdf")

fig.show()


### 4.1.3 Market Share by Parent Country

In [433]:
country_share = (
    df.groupby(["Year", "ParentCountry"], as_index=False)["Registrations"]
      .sum()
)

country_share["MarketShare_%"] = (
    country_share.groupby("Year")["Registrations"]
                 .transform(lambda x: 100 * x / x.sum())
)
country_share.sort_values(["Year", "MarketShare_%"], ascending=[True, False])

# --- Sort data for visual clarity ---
country_share = country_share.sort_values(["Year", "MarketShare_%"], ascending=[True, False])

# --- Line chart for market share trends ---
fig = px.line(
    country_share,
    x="Year",
    y="MarketShare_%",
    color="ParentCountry",
    markers=True,
    title="Market Share of Vehicle Registrations by Parent Country",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Market Share (%)",
    legend_title="Parent Country",
    template="plotly_white",
    width=900,
    height=550
)

# --- Save outputs ---
fig.write_image(FIG_DIR / "country_share_trends.pdf")

fig.show()


### 4.1.4 Herfindahl–Hirschman Index (HHI) — Market Concentration by Year

In [434]:
# HHI = sum of squared market shares (%)
hhi = (
    country_share.groupby("Year", as_index=False)["MarketShare_%"]
                 .apply(lambda x: (x**2).sum())
                 .rename(columns={"MarketShare_%": "HHI"})
)
hhi["HHI_Index"] = hhi["HHI"] / 10000  # normalize to 0–1 range
hhi


# --- Calculate HHI ---
hhi = (
    country_share.groupby("Year", as_index=False)["MarketShare_%"]
                 .apply(lambda x: (x**2).sum())
                 .rename(columns={"MarketShare_%": "HHI"})
)
hhi["HHI_Index"] = hhi["HHI"] / 10000  # normalize to 0–1

# --- Plot HHI over time ---
fig = px.line(
    hhi,
    x="Year",
    y="HHI_Index",
    markers=True,
    title="Market Concentration (HHI Index) Over Time",
    color_discrete_sequence=["crimson"]
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="HHI Index (0–1 scale)",
    template="plotly_white",
    width=850,
    height=500
)

# --- Save outputs ---
fig.write_html(FIG_DIR / "hhi_trend.html")
fig.write_image(FIG_DIR / "hhi_trend.pdf")

fig.show()


### 4.1.5 EV Penetration Over Time (Electric vs Total)

In [435]:
ev_penetration = (
    df.groupby(["Year", "Fuel Group"], as_index=False)["Registrations"]
      .sum()
      .pivot(index="Year", columns="Fuel Group", values="Registrations")
      .fillna(0)
)

ev_penetration["EV_Share_%"] = (
    100 * ev_penetration["Electric"] / ev_penetration.sum(axis=1)
)
ev_penetration = ev_penetration.reset_index()
ev_penetration[["Year", "EV_Share_%"]]

# --- Compute EV share ---
ev_penetration = (
    df.groupby(["Year", "Fuel Group"], as_index=False)["Registrations"]
      .sum()
      .pivot(index="Year", columns="Fuel Group", values="Registrations")
      .fillna(0)
)

ev_penetration["EV_Share_%"] = (
    100 * ev_penetration["Electric"] / ev_penetration.sum(axis=1)
)

ev_penetration = ev_penetration.reset_index()[["Year", "EV_Share_%"]]

# --- Plot ---
fig = px.line(
    ev_penetration,
    x="Year",
    y="EV_Share_%",
    markers=True,
    title="Electric Vehicle (EV) Penetration Over Time",
    color_discrete_sequence=["green"]
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="EV Share (%)",
    template="plotly_white",
    width=850,
    height=500
)

# --- Save ---
fig.write_image(FIG_DIR / "ev_penetration.pdf")

fig.show()

### 4.1.6 Top Parent Countries by Market Share (Last Available Year)

In [436]:
latest_year = df["Year"].max()
top_countries = (
    country_share[country_share["Year"] == latest_year]
    .sort_values("MarketShare_%", ascending=False)
    .head(10)
)
top_countries

# --- Compute latest year & filter top 10 ---
latest_year = df["Year"].max()
top_countries = (
    country_share[country_share["Year"] == latest_year]
    .sort_values("MarketShare_%", ascending=False)
    .head(10)
)

# --- Bar chart ---
fig = px.bar(
    top_countries,
    x="ParentCountry",
    y="MarketShare_%",
    text=top_countries["MarketShare_%"].round(2).astype(str) + "%",
    title=f"Top 10 Countries by Market Share — {latest_year}",
    color="ParentCountry",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Country of Origin",
    yaxis_title="Market Share (%)",
    template="plotly_white",
    width=850,
    height=500,
    showlegend=False
)

# --- Save outputs ---
fig.write_html(FIG_DIR / f"top_countries_{latest_year}.html")
fig.write_image(FIG_DIR / f"top_countries_{latest_year}.pdf")

fig.show()


### 4.1.7 Compound Annual Growth Rate (CAGR)

In [437]:
y0, y1 = df["Year"].min(), df["Year"].max()
r0 = annual_total.loc[annual_total["Year"] == y0, "Registrations"].values[0]
r1 = annual_total.loc[annual_total["Year"] == y1, "Registrations"].values[0]

CAGR = ((r1 / r0) ** (1 / (y1 - y0)) - 1) * 100
print(f"CAGR ({y0}-{y1}): {CAGR:.2f}%")

# --- Compute CAGR ---
y0, y1 = df["Year"].min(), df["Year"].max()
r0 = annual_total.loc[annual_total["Year"] == y0, "Registrations"].values[0]
r1 = annual_total.loc[annual_total["Year"] == y1, "Registrations"].values[0]

CAGR = ((r1 / r0) ** (1 / (y1 - y0)) - 1) * 100
print(f"CAGR ({y0}-{y1}): {CAGR:.2f}%")

# --- Optional: quick mini-plot for CAGR trend ---
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=annual_total["Year"],
    y=annual_total["Registrations"],
    mode="lines+markers",
    name="Registrations",
    line=dict(color="steelblue")
))

fig.add_annotation(
    x=y1, y=r1,
    text=f"CAGR {CAGR:.2f}%",
    showarrow=True,
    arrowhead=2,
    ax=-50, ay=-40,
    font=dict(size=12, color="crimson")
)

fig.update_layout(
    title=f"Annual Vehicle Registrations ({y0}–{y1}) — CAGR: {CAGR:.2f}%",
    xaxis_title="Year",
    yaxis_title="Registrations",
    template="plotly_white",
    width=850,
    height=500
)

fig.write_html(FIG_DIR / f"annual_registrations_cagr_{y0}_{y1}.html")
fig.write_image(FIG_DIR / f"annual_registrations_cagr_{y0}_{y1}.pdf")

fig.show()


CAGR (2000-2024): 7.71%
CAGR (2000-2024): 7.71%


### 4.1.8 Top 10 Brands by Total Registrations

In [438]:
top_makes = (
    df.groupby("Make", as_index=False)["Registrations"]
      .sum()
      .sort_values("Registrations", ascending=False)
      .head(10)
)
top_makes

# --- Aggregate top makes ---
top_makes = (
    df.groupby("Make", as_index=False)["Registrations"]
      .sum()
      .sort_values("Registrations", ascending=False)
      .head(10)
)

# --- Compute % share (optional) ---
total_reg = top_makes["Registrations"].sum()
top_makes["MarketShare_%"] = 100 * top_makes["Registrations"] / total_reg

# --- Plot bar chart ---
fig = px.bar(
    top_makes,
    x="Make",
    y="Registrations",
    text=top_makes["MarketShare_%"].round(1).astype(str) + "%",
    title="Top 10 Car Manufacturers by Total Registrations",
    color="Make",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Manufacturer (Make)",
    yaxis_title="Total Registrations",
    template="plotly_white",
    width=900,
    height=500,
    showlegend=False
)

# --- Save outputs ---
fig.write_image(FIG_DIR / "top_makes.pdf")

fig.show()

### 4.1.9 Chinese Brand Composition in the UK (Bar Plot Over Years)

In [ ]:
# --- Filter dataset to Chinese parent companies ---
df_china = df_veh_raw[
    (df_veh_raw["ParentCountry"] == "China") & (df_veh_raw["Year"] < 2025)
].copy()

# --- Aggregate registrations by Year × Make ---
china_comp = (
    df_china.groupby(["Year", "Make"], as_index=False)["Registrations"]
            .sum()
            .sort_values(["Year", "Registrations"], ascending=[True, False])
)

# --- Optional: keep top N brands for each year (rest -> 'Other Chinese') ---
top_brands = (
    china_comp.groupby("Year")["Registrations"]
              .apply(lambda x: x.nlargest(6).index)
              .explode()
)
china_comp["Make"] = china_comp.apply(
    lambda r: r["Make"] if r.name in top_brands.values else "Other Chinese", axis=1
)

china_comp = (
    china_comp.groupby(["Year", "Make"], as_index=False)["Registrations"]
              .sum()
)

# --- Plotly bar chart ---
fig = px.bar(
    china_comp,
    x="Year",
    y="Registrations",
    color="Make",
    title="Chinese Brand Composition in the UK Market (by Year)",
    text_auto=".2s",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.update_layout(
    barmode="stack",        # or "group" if you prefer side-by-side bars
    xaxis_title="Year",
    yaxis_title="Registrations",
    legend_title_text="Chinese Brand",
    template="plotly_white"
)

fig.write_image(FIG_DIR / "chinese_brand_comp.pdf")

fig.show()

### 4.1.10 China vs Germany in the UK

In [440]:
# Exclude 2025
df_market = df_veh_raw[df_veh_raw["Year"] < 2025].copy()

# Filter to just Chinese & German parent countries
df_cg = df_market[df_market["ParentCountry"].isin(["China", "Germany"])]

# Group by Year & ParentCountry
cg_trend = (
    df_cg.groupby(["Year", "ParentCountry"], as_index=False)["Registrations"]
         .sum()
)

# Compute market share of each within each year
cg_trend["Share_%"] = (
    cg_trend.groupby("Year")["Registrations"]
            .transform(lambda x: 100 * x / x.sum())
)

# --- Plot: Registration trend ---
fig = px.line(
    cg_trend,
    x="Year",
    y="Registrations",
    color="ParentCountry",
    title="UK Car Registrations: Chinese vs German Automakers",
    markers=True,
    color_discrete_sequence=["red", "steelblue"]
)
fig.update_layout(template="plotly_white", yaxis_title="Registrations")
fig.write_image(FIG_DIR / "reg_china_v_german_trend.pdf")

# --- Plot: Market share over time ---
fig2 = px.area(
    cg_trend,
    x="Year",
    y="Share_%",
    color="ParentCountry",
    title="Market Share in UK: Chinese vs German Automakers",
    color_discrete_sequence=["red", "steelblue"]
)
fig2.update_layout(template="plotly_white", yaxis_title="Market Share (%)")
fig.write_image(FIG_DIR / "market_share_china_v_german_trend.pdf")

fig.show()

# 5 Discussion Outputs

### 5.0.1 Setup & helpers

In [441]:
# Base working copy (exclude 2025)
df = df_veh_raw[df_veh_raw["Year"] < 2025].copy()

# Electric filter helper (uses your FuelGroup, but falls back if missing)
is_electric = df.get("Fuel Group", pd.Series(index=df.index, dtype=str)).eq("Electric")
if is_electric.isnull().all():
    # fallback if FuelGroup not present; expand patterns as needed
    is_electric = df["Fuel"].str.contains("electric", case=False, na=False)

# EV registrations per year
ev_by_year = (
    df.loc[is_electric]
      .groupby("Year", as_index=False)["Registrations"].sum()
      .rename(columns={"Registrations":"EV_Registrations"})
)


# --- 1) Load as strings to avoid silent coercion ---
fuel_raw = pd.read_csv(IN_DIR / "fuel_data.csv", dtype=str).rename(columns=str.strip)
fuel_raw.columns = [c.strip() for c in fuel_raw.columns]

# Expect columns like: "Year" and "Amount" (but allow variants)
label_col = next((c for c in fuel_raw.columns if c.lower().startswith("year")), None)
amount_col = next((c for c in fuel_raw.columns if "amount" in c.lower()), None)
if label_col is None or amount_col is None:
    raise ValueError(f"Couldn't find Year/Amount columns in {fuel_raw.columns.tolist()}")

# --- 2) Clean Amount -> numeric (remove currency, commas, NBSP, etc.) ---
amt = (fuel_raw[amount_col]
       .astype(str)
       .str.replace("\xa0", " ", regex=False)            # NBSP
       .str.replace(r"[£,]", "", regex=True)             # currency/commas
       .str.replace(r"[^\d\.\-]", "", regex=True)        # keep digits . -
       .str.strip())
fuel_raw["Amount_num"] = pd.to_numeric(amt, errors="coerce")

# --- 3) Normalise label column ---
lab = (fuel_raw[label_col]
       .astype(str)
       .str.replace("\xa0", " ", regex=False)
       .str.replace(r"\s+", " ", regex=True)
       .str.strip()
)

fuel_raw["Label"] = lab

# Identify row type
is_annual   = fuel_raw["Label"].str.fullmatch(r"\d{4}", na=False)
is_quarter  = fuel_raw["Label"].str.fullmatch(r"(\d{4}) Q([1-4])", na=False)
is_monthly  = fuel_raw["Label"].str.fullmatch(r"(\d{4}) ([A-Za-z]{3})", na=False)

# --- 4) Parse each type into Year ---
fuel_raw["Year"] = np.nan

# annual
fuel_raw.loc[is_annual, "Year"] = fuel_raw.loc[is_annual, "Label"].str.extract(r"(\d{4})").astype(int)

# quarterly
tmp_q = fuel_raw.loc[is_quarter, "Label"].str.extract(r"(?P<Y>\d{4}) Q(?P<Q>[1-4])")
fuel_raw.loc[is_quarter, "Year"] = tmp_q["Y"].astype(int)

# monthly
# use datetime to be robust to month abbreviations (JAN, FEB, …)
m_parse = pd.to_datetime(fuel_raw.loc[is_monthly, "Label"], format="%Y %b", errors="coerce")
fuel_raw.loc[is_monthly, "Year"] = m_parse.dt.year

fuel_raw["Year"] = fuel_raw["Year"].astype("Int64")

# --- 5) Aggregate to YEARLY FuelDuty (flow -> sum) at highest available granularity ---
annual_series   = (fuel_raw.loc[is_annual]
                   .groupby("Year", as_index=False)["Amount_num"].sum()
                   .rename(columns={"Amount_num": "Annual_total"}))

quarterly_year  = (fuel_raw.loc[is_quarter]
                   .groupby("Year", as_index=False)["Amount_num"].sum()
                   .rename(columns={"Amount_num": "Quarter_sum"}))

monthly_year    = (fuel_raw.loc[is_monthly]
                   .groupby("Year", as_index=False)["Amount_num"].sum()
                   .rename(columns={"Amount_num": "Month_sum"}))

# Merge and pick priority: monthly > quarterly > annual
fuel_year = (pd.DataFrame({"Year": pd.unique(fuel_raw["Year"].dropna().astype(int))})
             .merge(monthly_year,  on="Year", how="left")
             .merge(quarterly_year, on="Year", how="left")
             .merge(annual_series,  on="Year", how="left"))

fuel_year["FuelDuty"] = fuel_year["Month_sum"].where(fuel_year["Month_sum"].notna(),
                          fuel_year["Quarter_sum"].where(fuel_year["Quarter_sum"].notna(),
                          fuel_year["Annual_total"]))

fuel_year = (fuel_year[["Year", "FuelDuty"]]
             .dropna(subset=["Year"])
             .sort_values("Year")
             .reset_index(drop=True))

# Optional: restrict to analysis window (e.g., align with df_veh_raw and exclude 2025)
fuel_year = fuel_year[(fuel_year["Year"] >= df_veh_raw["Year"].min()) & (fuel_year["Year"] < 2025)]
fuel_year.head(12)

,Year,FuelDuty
30,2000,0.0
31,2001,0.0
32,2002,0.0
33,2003,0.0
34,2004,0.0
35,2005,0.0
36,2006,0.0
37,2007,0.0
38,2008,0.0
39,2009,0.0


### 5.1.2 Competition Structure: HHI pre- vs post-Chinese entry (EV segment)

In [442]:
# Define Chinese entry year: first year with Chinese EV registrations > 0
ev_china = df.loc[is_electric & (df["ParentCountry"]=="China")]
if ev_china.empty:
    raise ValueError("No Chinese EV entries found in df (ParentCountry == 'China').")

entry_year = int(ev_china.groupby("Year")["Registrations"].sum().loc[lambda s: s>0].index.min())

# Compute HHI (ParentCountry shares within EVs) by year
ev_parent = (
    df.loc[is_electric]
      .groupby(["Year","ParentCountry"], as_index=False)["Registrations"].sum()
)
ev_parent["Share_%"] = ev_parent.groupby("Year")["Registrations"].transform(lambda x: 100*x/x.sum())

hhi_by_year = (
    ev_parent.groupby("Year", as_index=False)["Share_%"]
             .apply(lambda s: (s**2).sum())
             .rename(columns={"Share_%":"HHI"})
)
hhi_by_year["HHI_Index"] = hhi_by_year["HHI"]/10000.0

pre_hhi  = hhi_by_year.loc[hhi_by_year["Year"] <  entry_year, "HHI_Index"].mean()
post_hhi = hhi_by_year.loc[hhi_by_year["Year"] >= entry_year, "HHI_Index"].mean()

print(f"Chinese EV entry year: {entry_year}")
print(f"Average HHI pre-entry:  {pre_hhi:.3f}")
print(f"Average HHI post-entry: {post_hhi:.3f}")


Chinese EV entry year: 2000
Average HHI pre-entry:  nan
Average HHI post-entry: 0.715


### 5.1.3 Market Entry Timing for each Chinese brand

In [443]:
china_brands = (
    df.loc[df["ParentCountry"]=="China"]
      .groupby(["Make","Year"], as_index=False)["Registrations"].sum()
)
first_year = (china_brands[china_brands["Registrations"]>0]
             .groupby("Make", as_index=False)["Year"].min()
             .rename(columns={"Year":"FirstYear_CHN"}))
first_year.sort_values("FirstYear_CHN")


,Make,FirstYear_CHN
29,MG,2000
25,LONDON TAXIS INT.,2000
20,LDV,2000
34,PROTON,2000
37,ROVER,2000
7,EASY RIDER,2000
14,JIALING,2000
26,LOTUS,2000
44,SMART,2003
24,LONCIN,2004


### 5.1.4 Product Mix Shift: BEV vs Hybrids ratio over time

In [444]:
# Classify EV detail using your Fuel labels (adjust patterns as needed)
tmp = df.loc[df["Fuel"].notna(), ["Year","Fuel","Registrations"]].copy()
tmp["EV_Type"] = np.select(
    [
        tmp["Fuel"].str.contains("battery electric", case=False, na=False),
        tmp["Fuel"].str.contains("plug[- ]?in hybrid", case=False, na=False) |
        tmp["Fuel"].str.contains("hybrid electric", case=False, na=False)
    ],
    ["BEV", "HybridEV"],
    default="Other"
)

mix = (
    tmp[tmp["EV_Type"].isin(["BEV","HybridEV"])]
      .groupby(["Year","EV_Type"], as_index=False)["Registrations"].sum()
      .pivot(index="Year", columns="EV_Type", values="Registrations")
      .fillna(0)
)

mix["BEV_to_Hybrid_Ratio"] = np.where(mix["HybridEV"]>0, mix["BEV"]/mix["HybridEV"], np.nan)
mix.reset_index()

# ---------- 1) Stacked bars: BEV vs HybridEV registrations ----------
mix_plot = mix.reset_index()  # Year back to column
fig = px.bar(
    mix_plot,
    x="Year",
    y=["BEV", "HybridEV"],
    title="BEV vs HybridEV Registrations by Year",
    labels={"value":"Registrations", "variable":"EV Type"},
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig.update_layout(barmode="stack", template="plotly_white", width=900, height=520)
fig.write_html(FIG_DIR / "bev_vs_hybrid_stacked.html")
fig.write_image(FIG_DIR / "bev_vs_hybrid_stacked.pdf")
fig.show()

# ---------- 2) % shares over time (area) ----------
mix_share = mix_plot.copy()
total = (mix_share["BEV"] + mix_share["HybridEV"]).replace(0, np.nan)
mix_share["BEV_%"] = 100 * mix_share["BEV"] / total
mix_share["HybridEV_%"] = 100 * mix_share["HybridEV"] / total

fig2 = px.area(
    mix_share,
    x="Year",
    y=["BEV_%", "HybridEV_%"],
    title="BEV vs HybridEV Share of EV Registrations (%)",
    labels={"value":"Share (%)", "variable":"EV Type"},
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig2.update_layout(template="plotly_white", width=900, height=520, yaxis_ticksuffix="%")
fig2.write_html(FIG_DIR / "bev_vs_hybrid_share.html")
fig2.write_image(FIG_DIR / "bev_vs_hybrid_share.pdf")
fig2.show()

# ---------- 3) Ratio line: BEV / HybridEV ----------
ratio_plot = mix[["BEV_to_Hybrid_Ratio"]].reset_index()
fig3 = px.line(
    ratio_plot,
    x="Year",
    y="BEV_to_Hybrid_Ratio",
    markers=True,
    title="BEV to HybridEV Ratio Over Time",
    labels={"BEV_to_Hybrid_Ratio":"BEV / HybridEV"},
    color_discrete_sequence=["crimson"]
)
fig3.update_layout(template="plotly_white", width=900, height=520)
fig3.write_image(FIG_DIR / "bev_to_hybrid_ratio.pdf")
fig3.show()

### 5.1.5) Cross-market Correlation: UK registrations vs stock returns (Chinese & German)

In [445]:

# --- A) Pull daily prices for representative listed OEMs ---
tickers = {
    # China
    "BYD": "1211.HK", "XPeng": "XPEV", "NIO": "NIO", "Li Auto": "LI",
    # Germany
    "Volkswagen": "VOW3.DE", "BMW": "BMW.DE", "Mercedes": "MBG.DE"
}
px = yf.download(list(tickers.values()), period="5y", auto_adjust=True)["Close"]
px.columns = tickers.keys()

# Monthly returns by origin
monthly = px.resample("M").last().pct_change().dropna()
origin_map = {c: "China" if c in {"BYD","XPeng","NIO","Li Auto"} else "Germany" for c in monthly.columns}
ret_by_origin = monthly.groupby(axis=1, by=origin_map).mean().rename(columns={"China":"Ret_China","Germany":"Ret_Germany"})

# --- UK registrations by origin at monthly or at least annual frequency ---
# If you only have yearly registrations, compute yearly returns instead.
regs_year_origin = (
    df.groupby(["Year","ParentCountry"], as_index=False)["Registrations"].sum()
      .pivot(index="Year", columns="ParentCountry", values="Registrations")
      .rename(columns={"China":"Reg_China","Germany":"Reg_Germany"})
      .dropna(how="all")
)

# Convert stock monthly returns to yearly returns to align (geometric aggregation)
yr = ret_by_origin.resample("Y").apply(lambda x: (1+x).prod()-1)
yr.index = yr.index.year

# Merge and compute correlations
corr_df = regs_year_origin.merge(yr, left_index=True, right_index=True, how="inner")
corrs = corr_df.corr(numeric_only=True)

print("Correlation matrix (registrations vs stock returns, annual):")
print(corrs.loc[["Reg_China","Reg_Germany"], ["Ret_China","Ret_Germany"]])

[*********************100%***********************]  7 of 7 completed
/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/1701599885.py:12: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/1701599885.py:14: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



Correlation matrix (registrations vs stock returns, annual):
             Ret_China  Ret_Germany
Reg_China    -0.509277    -0.398522
Reg_Germany  -0.547089    -0.570532


/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/1701599885.py:26: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



# Calculating Efficient Portfolio Based on Stocks From Past Years

In [446]:
brand_to_tickers = {
    # Directly listed automakers
    "ASTON MARTIN": ["AML.L"],
    "BMW": ["BMW.DE"],
    "BYD": ["1211.HK"],
    "DAIMLER": ["MBG.DE"],                 # Mercedes-Benz Group
    "FERRARI": ["RACE"],
    "FISKER": ["FSR"],                     # (historically; status may vary)
    "FORD": ["F"],
    "GEELY": ["0175.HK"],                  # parent (for LEVC/others)
    "HONDA": ["7267.T"],
    "HYUNDAI": ["005380.KS"],
    "INFINITI": ["7201.T"],                # via Nissan
    "JAGUAR": ["TATAMOTORS.NS", "TTM"],    # via Tata Motors
    "JEEP": ["STLA"],                      # via Stellantis
    "KIA": ["000270.KS"],
    "LEXUS": ["7203.T"],                   # via Toyota
    "LOTUS": ["LOT"],                      # Lotus Technology Inc (Nasdaq)
    "MASERATI": ["STLA"],                  # via Stellantis
    "MAZDA": ["7261.T"],
    "MERCEDES": ["MBG.DE"],
    "MERCEDES-BENZ": ["MBG.DE"],
    "MG": ["600104.SS"],                   # via SAIC Motor (A-share)
    "MINI": ["BMW.DE"],                    # via BMW
    "MITSUBISHI": ["7211.T"],              # Mitsubishi Motors
    "NISSAN": ["7201.T"],
    "OMODA": [],                           # via Chery (not listed)
    "ORA": ["601633.SS", "2333.HK"],       # via Great Wall Motor
    "PEUGEOT": ["STLA"],                   # via Stellantis
    "POLESTAR": ["PSNY"],
    "PORSCHE": ["P911.DE"],                # Porsche AG
    "RENAULT": ["RNO.PA"],
    "ROLLS ROYCE": ["BMW.DE"],            # cars via BMW (RR.L is aerospace)
    "SEAT": ["VOW3.DE"],                   # via Volkswagen
    "SKODA": ["VOW3.DE"],                  # via Volkswagen
    "SSANGYONG": ["003620.KS"],            # KG Mobility (KRX:003620)
    "SUBARU": ["7270.T"],
    "SUZUKI": ["7269.T"],
    "TATA": ["TATAMOTORS.NS", "TTM"],
    "TESLA": ["TSLA"],
    "TOYOTA": ["7203.T"],
    "VAUXHALL": ["STLA"],                  # via Stellantis
    "VOLKSWAGEN": ["VOW3.DE"],
    "VOLVO": ["VOLCAR-B.ST"],              # Volvo Car AB (cars)
    "XPENG": ["XPEV"],
    "YAMAHA": ["7272.T"],

    # Chinese groups & related
    "GREAT WALL": ["601633.SS", "2333.HK"],
    "GWM": ["601633.SS", "2333.HK"],
    "LEAPMOTOR": ["9863.HK"],
    "MAXUS": ["600104.SS"],                # via SAIC
    "ORA": ["601633.SS", "2333.HK"],
    "SKYWELL": [],                         # private / not widely listed
    "TIANJIN": [],                         # ambiguous brand; skip

    # Parent group tickers for brand families
    "ABARTH": ["STLA"],                    # Stellantis
    "ALFA ROMEO": ["STLA"],                # Stellantis
    "CITROEN": ["STLA"],                   # Stellantis
    "DS": ["STLA"],                        # Stellantis
    "FIAT": ["STLA"],                      # Stellantis
    "OPEL": ["STLA"],                      # Stellantis
    "CUPRA": ["VOW3.DE"],                  # VW/SEAT
    "AUDI": ["VOW3.DE"],                   # VW group
    "BENTLEY": ["VOW3.DE"],                # VW group
    "LAMBORGHINI": ["VOW3.DE"],            # (not in your list; VW group)
    "PORSCHE": ["P911.DE"],

    # US/others
    "CADILLAC": ["GM"],                    # via General Motors
    "CHEVROLET": ["GM"],
    "HUMMER": ["GM"],

    # Truck / industrial parents (if you still want them)
    "DAF": ["PCAR"],                       # via PACCAR
    "KOMATSU": ["6301.T"],                 # industrial
    "HYSTER": ["HY"],                      # Hyster-Yale Materials Handling (check)
    "YALE": ["HY"],                        # same parent

    # Hard/Private/Defunct/Coachbuilders -> no ticker
    "AIXAM": [], "ALBION": [], "ARIEL": [], "AUSTIN": [],
    "AUSTIN HEALEY": [], "AUSTIN MORRIS": [], "CATERPILLAR": ["CAT"],  # not cars; industrial
    "CLARK": [], "COLEMAN MILNE": [], "COVENTRY CLIMAX": [], "DAEWOO": [],
    "DATSUN": ["7201.T"],                  # via Nissan
    "DFSK": ["0489.HK", "601127.SS"],      # Dongfeng (0489.HK), Seres (601127.SS)
    "ELECTRICARS": [], "EOS": [], "EVT": [],
    "GENESIS": ["005380.KS"],              # via Hyundai Motor
    "GINETTA": [], "HARBILT ELECTRIC": [], "HILLMAN": [],
    "INVACAR": [], "JENSEN": [], "JONWAY": [],
    "KGM": ["003620.KS"],                  # KG Mobility (ex-SsangYong)
    "LANSING LINDE": [],                   # forklift brand (KION AG: KGX.DE if needed)
    "LIGIER": [], "LONDON TAXIS INT.": ["0175.HK"],   # via Geely (LEVC)
    "LOTUS": ["LOT"], "MAHINDRA": ["M&M.NS"],         # Yahoo uses 'M&M.NS'
    "MCC": ["MBG.DE"],                     # Smart’s original company -> Mercedes
    "MIA": [], "MITSUBISHI FUSO": ["DTG.DE"],         # via Daimler Truck
    "MOKE AUTOMIBILI": [], "MORGAN": [], "MORRIS": [],
    "OTHER AGRIC/CONSTRUCTIONAL": [], "OTHER BRITISH": [],
    "OTHER COUNTRIES": [], "OTHER ELECTRIC": [],
    "OTHER GERMAN": [], "OTHER ITALIAN": [], "OTHER US/AUST/CAN": [],
    "PERODUA": [], "PIONEER": [], "PROTON": [], "RELIANT": [],
    "RENAULT TRUCKS": ["VOLV-B.ST"],       # via Volvo Group (trucks), NOT cars
    "REVA": [], "ROVER": [], "ROYAL ENFIELD": ["EICHERMOT.NS"],  # parent Eicher Motors
    "SECMA": [], "SMART": ["MBG.DE", "0175.HK"],      # JV Mercedes/Geely
    "SMITHS": [], "THINK": [], "THINK GLOBAL": [],
    "TRIUMPH": [], "TVR": [], "W+E": [], "WESTFIELD": []
}


In [447]:
def get_tickers(brands, mapping=brand_to_tickers):
    rows = []
    for b in brands:
        key = b.strip().upper()
        tickers = mapping.get(key, [])
        rows.append({"Brand": b, "Tickers": ", ".join(tickers) if tickers else ""})
    return pd.DataFrame(rows)

brands = [
    'ABARTH', 'AIXAM', 'ALBION', 'ALFA ROMEO', 'ARIEL', 'ASTON MARTIN',
    'AUDI', 'AUSTIN', 'AUSTIN HEALEY', 'AUSTIN MORRIS', 'BENTLEY',
    'BMW', 'BYD', 'CADILLAC', 'CATERPILLAR', 'CHARGE', 'CHEVROLET',
    'CITROEN', 'CLARK', 'COLEMAN MILNE', 'COVENTRY CLIMAX', 'CUPRA',
    'DACIA', 'DAEWOO', 'DAF', 'DAIMLER', 'DATSUN', 'DFSK', 'DS',
    'ELECTRICARS', 'EOS', 'EVT', 'FERRARI', 'FIAT', 'FISKER', 'FORD',
    'GENESIS', 'GINETTA', 'GREAT WALL', 'GWM', 'HARBILT ELECTRIC',
    'HILLMAN', 'HONDA', 'HUMMER', 'HYSTER', 'HYUNDAI', 'INFINITI',
    'INVACAR', 'JAGUAR', 'JEEP', 'JENSEN', 'JONWAY', 'KGM', 'KIA',
    'KOMATSU', 'KYMCO', 'LAND ROVER', 'LANSING LINDE', 'LEAPMOTOR',
    'LEXUS', 'LIGIER', 'LONDON TAXIS INT.', 'LOTUS', 'MAHINDRA',
    'MASERATI', 'MAXUS', 'MAZDA', 'MCC', 'MERCEDES', 'MERCEDES-BENZ',
    'MG', 'MIA', 'MINI', 'MITSUBISHI', 'MITSUBISHI FUSO',
    'MOKE AUTOMIBILI', 'MORGAN', 'MORRIS', 'NISSAN', 'OMODA', 'OPEL',
    'ORA', 'OTHER AGRIC/CONSTRUCTIONAL', 'OTHER BRITISH',
    'OTHER COUNTRIES', 'OTHER ELECTRIC', 'OTHER GERMAN',
    'OTHER ITALIAN', 'OTHER US/AUST/CAN', 'PERODUA', 'PEUGEOT',
    'PIONEER', 'POLESTAR', 'PORSCHE', 'PROTON', 'RELIANT', 'RENAULT',
    'RENAULT TRUCKS', 'REVA', 'ROLLS ROYCE', 'ROVER', 'ROYAL ENFIELD',
    'SEAT', 'SECMA', 'SKODA', 'SKYWELL', 'SMART', 'SMITHS',
    'SSANGYONG', 'SUBARU', 'SUZUKI', 'TATA', 'TESLA', 'THINK',
    'THINK GLOBAL', 'TIANJIN', 'TOYOTA', 'TRIUMPH', 'TVR', 'VAUXHALL',
    'VOLKSWAGEN', 'VOLVO', 'W+E', 'WESTFIELD', 'XPENG', 'YALE',
    'YAMAHA'
]

# 1) Build and clean ticker table from brands + mapping
ticker_df = (
    pd.DataFrame(
        [{"Brand": b, "Tickers": ", ".join(brand_to_tickers.get(b.upper().strip(), []))}
         for b in brands]
    )
    .assign(Brand=lambda d: d["Brand"].str.upper().str.strip())
    .loc[lambda d: d["Tickers"].fillna("").str.strip() != ""]
)

# 2) Explode tickers, attach country info from df_brand, and dedupe
ticker_meta = (
    ticker_df.assign(Ticker=ticker_df["Tickers"].str.split(","))
             .explode("Ticker")
             .assign(Ticker=lambda d: d["Ticker"].str.strip())
             .loc[lambda d: d["Ticker"] != ""]
             .merge(
                 df_brand.assign(Make=df_brand["Make"].str.upper().str.strip())[
                     ["Make", "Country", "ParentCompany", "ParentCountry"]
                 ],
                 left_on="Brand", right_on="Make", how="left"
             )
             .drop(columns="Make")
             .drop_duplicates(subset=["Brand", "Ticker"])
             .reset_index(drop=True)
)

# Optional: keep only tickers present in your returns matrix
# ticker_meta = ticker_meta[ticker_meta["Ticker"].isin(rets_m.columns)].reset_index(drop=True)

ticker_meta.head(20)


,Brand,Tickers,Ticker,Country,ParentCompany,ParentCountry
0,ABARTH,STLA,STLA,Italy,Stellantis N.V.,Netherlands
1,ALFA ROMEO,STLA,STLA,Italy,Stellantis N.V.,Netherlands
2,ASTON MARTIN,AML.L,AML.L,United Kingdom,Aston Martin Lagonda Global Holdings plc,United Kingdom
3,AUDI,VOW3.DE,VOW3.DE,Germany,Volkswagen AG,Germany
4,BENTLEY,VOW3.DE,VOW3.DE,United Kingdom,Volkswagen AG,Germany
5,BMW,BMW.DE,BMW.DE,Germany,BMW AG,Germany
6,BYD,1211.HK,1211.HK,China,"BYD Auto Co., Ltd.",China
7,CADILLAC,GM,GM,United States,General Motors,United States
8,CATERPILLAR,CAT,CAT,United States,Caterpillar Inc.,United States
9,CHEVROLET,GM,GM,United States,General Motors,United States


In [448]:
# ==== Efficient Frontier from ticker_df =======================================
# Requirements: pip install yfinance scipy plotly (or use matplotlib if you prefer)


# ------------------- Config -------------------
start_date = "2019-01-01"     # adjust as needed
end_date   = None             # None -> until today
rf_annual  = 0.03             # risk-free (annual). Change if you have a UK gilt rate.
freq_per_year = 12            # monthly returns
allow_short = False           # set True for unconstrained frontier

# ------------------- 1) Build ticker list -------------------
# Use the FIRST listed ticker per brand; drop empties & duplicates
tickers = (ticker_df["Tickers"]
           .dropna()
           .astype(str)
           .str.split(",")
           .str[0]
           .str.strip()
           .replace("", np.nan)
           .dropna()
           .unique()
           .tolist())

assert len(tickers) > 1, "Need at least 2 tradable tickers."

# Optional: mapping Brand -> chosen ticker (for labeling later)
brand_to_one_ticker = (ticker_df.assign(Ticker=ticker_df["Tickers"].str.split(",").str[0].str.strip())
                       .dropna(subset=["Ticker"])
                       .drop_duplicates(subset=["Ticker"])
                       .set_index("Ticker")["Brand"].to_dict())

# ------------------- 2) Download prices & compute returns -------------------
prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True, progress=False)["Close"]
if isinstance(prices, pd.Series):  # if only one ticker sneaks through
    prices = prices.to_frame()

# Resample to monthly, compute pct returns
rets_m = prices.resample("M").last().pct_change().dropna(how="all").dropna(axis=1)
tickers = rets_m.columns.tolist()  # keep only those with data

# Annualize mean & covariance (monthly -> annual)
mu = rets_m.mean() * freq_per_year
Sigma = rets_m.cov() * freq_per_year
n = len(tickers)

# ------------------- 3) Helpers for optimization -------------------
ones = np.ones(n)

def portfolio_perf(w, mu, Sigma):
    ret = float(w @ mu)
    vol = float(np.sqrt(w @ Sigma @ w))
    return ret, vol

def min_var_given_return(target_ret, mu, Sigma, long_only=True):
    # Minimize w' Σ w, s.t. sum w = 1, mu·w = target_ret, and (optional) w>=0
    def objective(w): return w @ Sigma @ w
    cons = (
        {"type": "eq", "fun": lambda w: w.sum() - 1.0},
        {"type": "eq", "fun": lambda w: w @ mu - target_ret}
    )
    bounds = None if not long_only else [(0.0, 1.0)] * n
    w0 = ones / n
    res = minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=cons)
    return res.x if res.success else None

def min_variance(mu, Sigma, long_only=True):
    # Minimize variance with sum w = 1 (no target return)
    def objective(w): return w @ Sigma @ w
    cons = ({"type": "eq", "fun": lambda w: w.sum() - 1.0},)
    bounds = None if not long_only else [(0.0, 1.0)] * n
    w0 = ones / n
    res = minimize(objective, w0, method="SLSQP", bounds=bounds, constraints=cons)
    return res.x if res.success else None

def max_sharpe(mu, Sigma, rf=0.0, long_only=True):
    # Maximize Sharpe -> minimize negative Sharpe
    def neg_sharpe(w):
        ret, vol = portfolio_perf(w, mu, Sigma)
        return -(ret - rf) / (vol + 1e-12)
    cons = ({"type": "eq", "fun": lambda w: w.sum() - 1.0},)
    bounds = None if not long_only else [(0.0, 1.0)] * n
    w0 = ones / n
    res = minimize(neg_sharpe, w0, method="SLSQP", bounds=bounds, constraints=cons)
    return res.x if res.success else None

# ------------------- 4) Compute frontier curve -------------------
# Target returns from (near) min to max achievable
min_mu, max_mu = float(mu.min()), float(mu.max())
targets = np.linspace(min_mu + 1e-6, max_mu - 1e-6, 40)

frontier = []
weights_list = []
for tr in targets:
    w = min_var_given_return(tr, mu, Sigma, long_only=not allow_short is False)  # long-only by default
    if w is None: 
        continue
    r, v = portfolio_perf(w, mu, Sigma)
    frontier.append({"Return": r, "Volatility": v})
    weights_list.append(w)

frontier_df = pd.DataFrame(frontier)

# Key portfolios
w_minvar = min_variance(mu, Sigma, long_only=not allow_short is False)
w_tan    = max_sharpe(mu, Sigma, rf=rf_annual, long_only=not allow_short is False)

r_min, v_min = portfolio_perf(w_minvar, mu, Sigma)
r_tan, v_tan = portfolio_perf(w_tan,    mu, Sigma)
sh_tan = (r_tan - rf_annual) / v_tan

# ------------------- 5) Plot (interactive) -------------------
import plotly.express as px

fig = px.scatter(frontier_df, x="Volatility", y="Return", title="Efficient Frontier (Annualized)")
fig.update_traces(mode="lines+markers")

# Mark min-variance and tangency
fig.add_trace(go.Scatter(x=[v_min], y=[r_min], mode="markers", name="Min-Var", marker=dict(size=10)))
fig.add_trace(go.Scatter(x=[v_tan], y=[r_tan], mode="markers", name=f"Tangent (Sharpe={sh_tan:.2f})", marker=dict(size=10)))

# (Optional) Capital Market Line
x_cml = np.linspace(0, max(frontier_df["Volatility"].max(), v_tan)*1.1, 50)
y_cml = rf_annual + ( (r_tan - rf_annual) / v_tan ) * x_cml
fig.add_trace(go.Scatter(x=x_cml, y=y_cml, mode="lines", name="CML", line=dict(dash="dash")))

fig.update_layout(xaxis_title="Volatility (σ, annualized)", yaxis_title="Return (μ, annualized)", template="plotly_white")
fig.write_html(OUT_DIR / "efficient_frontier.html")
fig.write_image(FIG_DIR / "efficient_frontier.pdf")
fig.show()

# ------------------- 6) Export weights -------------------
def weights_df(w, name):
    d = pd.DataFrame({"Ticker": tickers, "Brand": [brand_to_one_ticker.get(t, "") for t in tickers], "Weight": w})
    d = d[d["Weight"].abs() > 1e-6].sort_values("Weight", ascending=False).reset_index(drop=True)
    d.to_csv(OUT_DIR / f"weights_{name}.csv", index=False)
    return d

weights_minvar = weights_df(w_minvar, "min_variance")
weights_tan    = weights_df(w_tan,    "tangency")

print("Min-Variance portfolio:")
print(weights_minvar.head(20))
print("\nTangency (max Sharpe) portfolio:")
print(weights_tan.head(20))
print(f"\nTangency: Return={r_tan:.2%}, Volatility={v_tan:.2%}, Sharpe={sh_tan:.2f} (rf={rf_annual:.2%})")



1 Failed download:
['FSR']: YFTzMissingError('possibly delisted; no timezone found')
/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/560136644.py:39: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/560136644.py:39: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



Min-Variance portfolio:
          Ticker           Brand    Weight
0   EICHERMOT.NS   ROYAL ENFIELD  0.248216
1           RACE         FERRARI  0.242405
2      600104.SS           MAXUS  0.224260
3           PCAR             DAF  0.198308
4        VOW3.DE            AUDI  0.181881
5         7267.T           HONDA  0.175975
6        1211.HK             BYD  0.156760
7         7211.T      MITSUBISHI  0.141782
8      VOLV-B.ST  RENAULT TRUCKS  0.129760
9         7270.T          SUBARU  0.112241
10        MBG.DE         DAIMLER  0.082648
11           CAT     CATERPILLAR  0.062001
12     005380.KS         GENESIS  0.060359
13            HY          HYSTER  0.035023
14       0489.HK            DFSK  0.015877
15        7272.T          YAMAHA  0.014510
16         AML.L    ASTON MARTIN  0.006742
17        7203.T           LEXUS  0.005493
18        M&M.NS        MAHINDRA  0.002349
19     003620.KS             KGM  0.000842

Tangency (max Sharpe) portfolio:
           Ticker           Brand    We

In [449]:
# --- Use your monthly returns from earlier ---
# If you followed the efficient frontier code, just reuse:
rets_m = prices.resample("M").last().pct_change().dropna(how="all").dropna(axis=1)

# --- Compute correlation matrix ---
corr_df = rets_m.corr()

# --- Plot with Plotly (interactive heatmap) ---
fig = px.imshow(
    corr_df,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    zmin=-1,
    zmax=1,
    title="Asset Correlation Matrix"
)
fig.update_layout(
    width=800,
    height=700,
    xaxis_title="Assets",
    yaxis_title="Assets",
    template="plotly_white"
)
fig.show()

# --- Optional 4️⃣: Save correlation matrix ---
corr_df.to_csv("outputs/asset_correlation_matrix.csv")

/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/1737317200.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/1737317200.py:3: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



### 1 Build Ticker → Country mapping (from your brand table)

In [450]:

# brand_to_one_ticker: {ticker: brand}; ensure uppercase brands
ticker_brand = (pd.Series(brand_to_one_ticker, name="Brand")
                .rename_axis("Ticker").reset_index())
ticker_brand["Brand"] = ticker_brand["Brand"].str.upper().str.strip()

# df_brand: has columns ["Make","Country","ParentCompany","ParentCountry"]
brand_origin = df_brand[["Make","ParentCountry"]].copy()
brand_origin["Make"] = brand_origin["Make"].str.upper().str.strip()

# join to get each ticker's country of (parent) origin
ticker_meta = (ticker_brand.merge(brand_origin, left_on="Brand", right_on="Make", how="left")
                          .drop(columns=["Make"])
                          .rename(columns={"ParentCountry":"Country"}))

# keep only tickers present in your return matrix
tickers_in_data = set(rets_m.columns)
ticker_meta = ticker_meta[ticker_meta["Ticker"].isin(tickers_in_data)].dropna(subset=["Country"])
assert not ticker_meta.empty, "No tickers mapped to countries. Check brand names & df_brand."


### 2) Country portfolios (equal-weighted within each country)

In [451]:
# --- 0) From your ticker_meta (must have columns: Tickers, Country) ---
ticker_meta = ticker_meta.dropna(subset=["Ticker","Country"]).copy()
tickers = sorted(ticker_meta["Ticker"].unique().tolist())
assert len(tickers) >= 2, "Need at least 2 tickers."

# --- 1) Pull prices and build monthly returns ---
prices = yf.download(tickers, period="5y", auto_adjust=True, progress=False)["Close"]
if isinstance(prices, pd.Series):  # if only 1 ticker returned
    prices = prices.to_frame()
rets_m = prices.resample("M").last().pct_change().dropna(how="all").dropna(axis=1)
tickers = rets_m.columns.tolist()  # keep only tickers with data

# --- 2) Keep only mapped tickers & build equal-weight country weights ---
tm = (ticker_meta[ticker_meta["Ticker"].isin(tickers)]
      .drop_duplicates(["Country","Ticker"]))

w_eq = (tm.groupby("Country")["Ticker"]
          .apply(lambda s: pd.Series(1.0/len(s), index=s.values))
          .rename("Weight").reset_index())
# --- Build weight matrix: Countries × Tickers ---
W = (
    w_eq.pivot(index="Country", columns="level_1", values="Weight")
        .reindex(columns=rets_m.columns, fill_value=0.0)
)

# --- Re-normalize so each country's weights sum to 1 ---
W = W.div(W.sum(axis=1), axis=0).fillna(0.0)


# --- 3) Country portfolios: monthly returns, stats, correlations ---
country_rets = rets_m.dot(W.T)          # index = month, cols = Country
freq = 12
country_stats = pd.DataFrame({
    "AnnReturn": country_rets.mean()*freq,
    "AnnVol": country_rets.std(ddof=1)*np.sqrt(freq)
})
country_stats["Sharpe(0%)"] = country_stats["AnnReturn"] / country_stats["AnnVol"]
country_corr = country_rets.corr()

# (Optional) save
country_stats.to_csv(OUT_DIR / "country_portfolio_stats.csv")
country_corr.to_csv(OUT_DIR / "country_portfolio_correlations.csv")

/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/3423980527.py:10: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [452]:
from scipy.optimize import minimize

mu = country_rets.mean()*freq
Sigma = country_rets.cov()*freq
countries = mu.index.tolist()
n = len(countries)
ones = np.ones(n)

def perf(w): r = float(w@mu.values); v = float(np.sqrt(w@Sigma.values@w)); return r, v
def min_var(bounds=(0,1)):
    cons = ({"type":"eq","fun":lambda w: w.sum()-1.0},)
    bnds = [bounds]*n if bounds else None
    res = minimize(lambda w: w@Sigma.values@w, ones/n, method="SLSQP", bounds=bnds, constraints=cons)
    return res.x
def max_sharpe(rf=0.03, bounds=(0,1)):
    def negS(w): r,v = perf(w); return -(r-rf)/(v+1e-12)
    cons = ({"type":"eq","fun":lambda w: w.sum()-1.0},)
    bnds = [bounds]*n if bounds else None
    res = minimize(negS, ones/n, method="SLSQP", bounds=bnds, constraints=cons)
    return res.x

w_min = min_var(bounds=(0,1))
w_tan = max_sharpe(rf=0.03, bounds=(0,1))

weights_min = pd.Series(w_min, index=countries, name="MinVar").sort_values(ascending=False)
weights_tan = pd.Series(w_tan, index=countries, name="Tangency").sort_values(ascending=False)

print(weights_min, weights_tan)

mu = country_rets.mean()*freq
Sigma = country_rets.cov()*freq
countries = mu.index.tolist()
n = len(countries)
ones = np.ones(n)

def perf(w): r = float(w@mu.values); v = float(np.sqrt(w@Sigma.values@w)); return r, v
def min_var(bounds=(0,1)):
    cons = ({"type":"eq","fun":lambda w: w.sum()-1.0},)
    bnds = [bounds]*n if bounds else None
    res = minimize(lambda w: w@Sigma.values@w, ones/n, method="SLSQP", bounds=bnds, constraints=cons)
    return res.x
def max_sharpe(rf=0.03, bounds=(0,1)):
    def negS(w): r,v = perf(w); return -(r-rf)/(v+1e-12)
    cons = ({"type":"eq","fun":lambda w: w.sum()-1.0},)
    bnds = [bounds]*n if bounds else None
    res = minimize(negS, ones/n, method="SLSQP", bounds=bnds, constraints=cons)
    return res.x

w_min = min_var(bounds=(0,1))
w_tan = max_sharpe(rf=0.03, bounds=(0,1))

weights_min = pd.Series(w_min, index=countries, name="MinVar").sort_values(ascending=False)
weights_tan = pd.Series(w_tan, index=countries, name="Tangency").sort_values(ascending=False)

print(weights_min, weights_tan)

Japan             3.047196e-01
Sweden            2.020657e-01
South Korea       1.657047e-01
India             1.365167e-01
Italy             1.042410e-01
China             8.675232e-02
Germany           4.457087e-17
France            7.299582e-18
United Kingdom    1.805874e-18
Netherlands       0.000000e+00
United States     0.000000e+00
Name: MinVar, dtype: float64 India             5.592951e-01
Japan             1.743774e-01
Sweden            1.278097e-01
Italy             8.007640e-02
China             5.844144e-02
United Kingdom    3.150258e-15
France            7.557973e-16
United States     7.470153e-16
South Korea       5.043166e-16
Germany           7.309962e-17
Netherlands       0.000000e+00
Name: Tangency, dtype: float64
Japan             3.047196e-01
Sweden            2.020657e-01
South Korea       1.657047e-01
India             1.365167e-01
Italy             1.042410e-01
China             8.675232e-02
Germany           4.457087e-17
France            7.299582e-18
United Kin

### 3) Correlation between country portfolios

In [453]:
country_corr = country_rets.corr()
country_corr.to_csv("outputs/country_portfolio_correlations.csv")
country_corr
# --- Compute and export correlation matrix ---
country_corr = country_rets.corr()
country_corr.to_csv(OUT_DIR / "country_portfolio_correlations.csv")

# --- Plot interactive heatmap ---
fig = px.imshow(
    country_corr,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    title="Correlation Between Country Portfolios",
)

fig.update_layout(
    template="plotly_white",
    width=900,
    height=700,
    xaxis_title="Country",
    yaxis_title="Country"
)

# --- Save visual outputs ---
fig.write_html(FIG_DIR / "country_corr_heatmap.html")
fig.write_image(FIG_DIR / "country_corr_heatmap.pdf")

fig.show()

### 4) (Optional) Rolling 12-month correlations between two countries

In [454]:
c1, c2 = "China", "Germany"   # change as needed
rolling_corr = (country_rets[c1].rolling(12).corr(country_rets[c2])
                               .dropna())
rolling_corr.to_csv("outputs/rolling_corr_12m_{}_{}.csv".format(c1, c2))


### 5) (Optional) Cap-weighted within each country (if you have market caps)

In [455]:
# ---  Ensure ticker_meta has Ticker + Country ---
tm = (ticker_meta.dropna(subset=["Ticker","Country"])
                  .assign(Ticker=lambda d: d["Ticker"].str.strip().str.upper())
                  .drop_duplicates(["Country","Ticker"]))

# --- Fetch market caps (if you don’t already have them) ---
caps = []
for t in tm["Ticker"].unique():
    try:
        info = yf.Ticker(t).info
        cap = info.get("marketCap", np.nan)
        caps.append({"Ticker": t, "MarketCap": cap})
    except Exception:
        pass
cap_df = pd.DataFrame(caps).dropna(subset=["MarketCap"])
print(f"✓ Retrieved {len(cap_df)} valid market caps out of {len(tm)} tickers.")

# --- Merge and compute within-country cap weights ---
w_cap = (
    tm.merge(cap_df, on="Ticker", how="inner")
      .groupby("Country")
      .apply(lambda g: g.assign(Weight=g["MarketCap"] / g["MarketCap"].sum()))
      .reset_index(drop=True)[["Country","Ticker","Weight"]]
)

# ---  Build weight matrix Countries × Tickers ---
W_cap = (
    w_cap.pivot(index="Country", columns="Ticker", values="Weight")
         .reindex(columns=rets_m.columns, fill_value=0.0)
)
W_cap = W_cap.div(W_cap.sum(axis=1), axis=0).fillna(0.0)

# ---  Compute country cap-weighted monthly returns ---
country_rets_cap = rets_m.dot(W_cap.T)
country_corr_cap = country_rets_cap.corr()

# ---  Optional: export ---
# country_rets_cap.to_csv(OUT_DIR / "country_rets_cap_weighted.csv")
# country_corr_cap.to_csv(OUT_DIR / "country_corr_cap_weighted.csv")

print("country_rets_cap shape:", country_rets_cap.shape)
print("country_corr_cap shape:", country_corr_cap.shape)



✓ Retrieved 34 valid market caps out of 34 tickers.
country_rets_cap shape: (60, 11)
country_corr_cap shape: (11, 11)


/var/folders/7x/pd53f7tx0jl5lyy8g87z1qqw0000gn/T/ipykernel_13578/3111319913.py:22: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



### 6) (Optional) Portfolio across countries with your own country weights

In [456]:
# --- Custom country blend: e.g., 60% Germany, 40% China ---
country_weights = pd.Series({"Germany": 0.6, "China": 0.4}, name="w")
country_weights = country_weights / country_weights.sum()

# Align weights to available countries in country_rets
valid_countries = [c for c in country_weights.index if c in country_rets.columns]
country_weights = country_weights.loc[valid_countries]

# Compute portfolio monthly returns
portfolio_rets = country_rets[valid_countries].mul(country_weights, axis=1).sum(axis=1)

# Optional: annualised stats
freq = 12
ann_ret = portfolio_rets.mean() * freq
ann_vol = portfolio_rets.std(ddof=1) * np.sqrt(freq)
sharpe = ann_ret / ann_vol

print(f"Custom Portfolio → Annualised Return={ann_ret:.2%}, Volatility={ann_vol:.2%}, Sharpe(0%)={sharpe:.2f}")



Custom Portfolio → Annualised Return=15.11%, Volatility=23.51%, Sharpe(0%)=0.64
